In [ ]:
!pip install pennylane

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 14.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 52.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.9/49.9 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.1/18.1 MB 53.5 MB/s eta 0:00:00


In [ ]:
import pennylane as qml
import pennylane.numpy as np
import matplotlib.pyplot as plt

In [76]:
def vec_check(vec,electron_num):
    """
    To check solution conserve the charge
    """
    for i,e in enumerate(vec):
        if np.abs(e) > 1e-8:
            e_occ=bin(i)[2:]
            lst=' '.join(e_occ).split(' ')
            cv=tuple(map(int, lst))
            e_num=sum(cv)
            if e_num!=electron_num:
                return False
    return True

def exact_ch_energy(num_electrons,symbol,geometry,charge):
    Ha=qml.qchem.molecular_hamiltonian(symbol, geometry, charge=charge,\
                                               basis="STO-3G",active_electrons=num_electrons)[0]
    H_matrix=qml.matrix(Ha)
    vals, vecs = np.linalg.eigh(H_matrix)

    inds=np.argsort(vals)
    eng=vals[inds]
    vec=vecs[:,inds]
    E_collect=[0,0,0,0]
    E_collect[0]=eng[0]
    print('The ground state energy with sz=0:',eng[0])

    sz=qml.qchem.spinz(len(Ha.wires))
    SZ=qml.matrix(sz)
    Stop=[False,False,False]

    elec_num=num_electrons
    for i in range(1,len(eng)): # Finding first excited state with -1 Sz
        Sz=vec[:,i].dot(SZ.dot(np.transpose(np.conjugate(vec[:,i]))))
        if np.abs(Sz) < 1e-5 and vec_check(vec[:,i],elec_num):
            print('The first excied state energy with sz=0:',eng[i])
            Stop[0]=True
            E_collect[1]=eng[i]
        elif np.abs(Sz-1) < 1e-5 and vec_check(vec[:,i],elec_num):
            print('The first excied state energy with sz=1:',eng[i])
            Stop[1]=True
            E_collect[2]=eng[i]
        elif np.abs(Sz+1) < 1e-5 and vec_check(vec[:,i],elec_num):
            print('The first excied state energy with sz=-1:',eng[i])
            Stop[2]=True
            E_collect[3]=eng[i]
        if np.all(Stop):
            break
    return E_collect


In [77]:
L=1.5
symbol=["H", "Li"]
geometry=np.array([0.0, 0.0, -L/2, 0.0, 0.0, L/2])

charge=0
num_electrons=2
exact_ch_energy(num_electrons,symbol,geometry,charge)

The ground state energy with sz=0: -7.626624465547493
The first excied state energy with sz=1: -7.522914627103037
The first excied state energy with sz=0: -7.522914627103034
The first excied state energy with sz=-1: -7.52291462710303


[-7.626624465547493, -7.522914627103034, -7.522914627103037, -7.52291462710303]

In [78]:
def depth(qnode):
    def _fn(*args, **kwargs):
        qnode.construct(args, kwargs)
        return qnode.qtape.get_depth()
    return _fn

In [75]:
def bit_to_vec(bitstring_dict,num_qubits):
    vec=np.zeros(2**num_qubits)
    amp=0
    for i in range(2**num_qubits):
        bitstr=bin(i)[2:]
        if len(bitstr) < num_qubits:
            bitstr=(num_qubits-len(bitstr))*'0'+bitstr

        if bitstr in bitstring_dict:
            vec[i]=bitstring_dict[bitstr]
            amp+=np.abs(bitstring_dict[bitstr])**2
    vec=vec/np.sqrt(amp)
    return vec

In [82]:
class vqe_X_ch():
    def __init__(self,state,num_electrons,symbol,geometry,charge) -> None:
        self.num_electrons=num_electrons
        self.symbol=symbol
        self.geometry=geometry
        self.charge=charge

        self.hamiltonian=self.Ch_hamiltonian(symbol,geometry,num_electrons,charge)
        self.num_qubits = len(self.hamiltonian.wires)
        self.sz=qml.qchem.spinz(self.num_qubits)
        self.singles, self.doubles = qml.qchem.excitations(self.num_electrons, self.num_qubits)
        self.state=bit_to_vec(state,self.num_qubits)
        H_matrix=qml.matrix(self.hamiltonian)
        H_sq=H_matrix.dot(H_matrix)

        self.H_sq_ob=qml.Hermitian(H_sq, wires=range(self.num_qubits))
        self.dev = qml.device("default.qubit", wires=self.num_qubits)
        self.cost_h_fn = qml.QNode(self.circuit, self.dev)
        self.Sz_fn = qml.QNode(self.circuit_sz, self.dev)
        self.cost_hsq_fn = qml.QNode(self.circuit_h_sq, self.dev)

    def Ch_hamiltonian(self,symbol,geometry,num_electrons,charge):
        return qml.qchem.molecular_hamiltonian(symbol, geometry, charge=charge,\
                                               basis="STO-3G",active_electrons=num_electrons)[0]

    def hf(self,electrons, num_qubits):
        return qml.qchem.hf_state(electrons=electrons, orbitals=num_qubits)

    def circuit(self,state,weights):
        qml.StatePrep(state, wires=range(self.num_qubits))
        for i in range(len(self.singles)):
            qml.SingleExcitation(weights[i+len(self.doubles)], wires=self.singles[i])
        for i in range(len(self.doubles)):
            qml.DoubleExcitation(weights[i], wires=self.doubles[i])
        return qml.expval(self.hamiltonian)

    def circuit_h_sq(self,state,weights):
        qml.StatePrep(state, wires=range(self.num_qubits))
        for i in range(len(self.singles)):
            qml.SingleExcitation(weights[i+len(self.doubles)], wires=self.singles[i])
        for i in range(len(self.doubles)):
            qml.DoubleExcitation(weights[i], wires=self.doubles[i])
        return qml.expval(self.H_sq_ob)

    def circuit_sz(self,state,weights):
        qml.StatePrep(state, wires=range(self.num_qubits))
        for i in range(len(self.singles)):
            qml.SingleExcitation(weights[i+len(self.doubles)], wires=self.singles[i])
        for i in range(len(self.doubles)):
            qml.DoubleExcitation(weights[i], wires=self.doubles[i])
        return qml.expval(self.sz)

    def cost_fn(self,params):
        self.cost_hsq_fn = qml.QNode(self.circuit_h_sq, self.dev)
        return np.sqrt(self.cost_hsq_fn(self.state,params)-self.cost_h_fn(self.state,params)**2)

    def run(self,epochs=400,stepsize=0.5,cov=1e-6,print_train=True):
        opt = qml.AdamOptimizer(stepsize=stepsize)
        self.weights =np.zeros(len(self.doubles + self.singles), requires_grad=True)
        i=0
        iter=[]
        cost_val=[]
        Lowest_cost=0
        self.best_weights=self.weights

        cov=cov
        E0_pev=0
        for _ in range(epochs):
            self.weights = opt.step(self.cost_fn, self.weights)
            iter.append(i)
            cost=self.cost_fn(self.weights)
            E0=self.cost_h_fn(self.state,self.weights)

            if Lowest_cost>cost:
                Lowest_cost=cost
                self.best_weights=self.weights
            cost_val.append(E0)

            i=i+1
            if print_train:
                print('iter:',i,' E0:',E0,' Sz:',self.Sz_fn(self.state,self.weights))
            if np.abs(E0-E0_pev)<cov:
                break

            E0_pev=E0
        self.E0=E0
        return E0


In [115]:
L=1.5
symbol=["H", "Li"]
geometry=np.array([0.0, 0.0, -L/2, 0.0, 0.0, L/2])

charge=0
num_electrons=2
e_trial_stat_sz0={'0101000000': 1}
vqe_x_test=vqe_X_ch(e_trial_stat_sz0,num_electrons,symbol,geometry,charge)

In [116]:
vqe_x_test.run(stepsize=0.5)

iter: 1  E0: -7.517692384719897  Sz: -1.0
iter: 2  E0: -7.514239380733699  Sz: -1.0
iter: 3  E0: -7.518771845976776  Sz: -0.9999999999999999
iter: 4  E0: -7.516220142977723  Sz: -1.0
iter: 5  E0: -7.517132356342593  Sz: -0.9999999999999998
iter: 6  E0: -7.5187728786711965  Sz: -0.9999999999999998
iter: 7  E0: -7.516836759338113  Sz: -1.0
iter: 8  E0: -7.5175301502878416  Sz: -1.0000000000000002
iter: 9  E0: -7.518863284232784  Sz: -1.0
iter: 10  E0: -7.517907378987171  Sz: -1.0
iter: 11  E0: -7.5181185987854855  Sz: -1.0
iter: 12  E0: -7.518862905438367  Sz: -0.9999999999999999
iter: 13  E0: -7.518041747034343  Sz: -1.0
iter: 14  E0: -7.517987215235522  Sz: -1.0
iter: 15  E0: -7.518809780165736  Sz: -1.0000000000000002
iter: 16  E0: -7.518558717096734  Sz: -0.9999999999999999
iter: 17  E0: -7.518395615383124  Sz: -1.0000000000000002
iter: 18  E0: -7.518845314955326  Sz: -1.0
iter: 19  E0: -7.518542371223495  Sz: -1.0
iter: 20  E0: -7.518298238182189  Sz: -1.0
iter: 21  E0: -7.518762852

tensor(-7.5188634, requires_grad=True)

In [108]:
class adapt_vqe_X_ch():
    def __init__(self,state,num_electrons,symbol,geometry,charge) -> None:
        self.num_electrons=num_electrons
        self.symbol=symbol
        self.geometry=geometry
        self.charge=charge

        self.hamiltonian=self.Ch_hamiltonian(symbol,geometry,num_electrons,charge)
        self.num_qubits = len(self.hamiltonian.wires)
        self.sz=qml.qchem.spinz(self.num_qubits)
        self.singles, self.doubles = qml.qchem.excitations(self.num_electrons, self.num_qubits)
        self.state=bit_to_vec(state,self.num_qubits)
        self.pool_operators=self.singles+self.doubles
        H_matrix=qml.matrix(self.hamiltonian)
        H_sq=H_matrix.dot(H_matrix)

        self.H_sq_ob=qml.Hermitian(H_sq, wires=range(self.num_qubits))
        self.dev = qml.device("default.qubit", wires=self.num_qubits)
        self.cost_h_fn = qml.QNode(self.circuit, self.dev, expansion_strategy="device")
        self.Sz_fn = qml.QNode(self.circuit_sz, self.dev)
        self.cost_hsq_fn = qml.QNode(self.circuit_h_sq, self.dev, expansion_strategy="device")

    def Ch_hamiltonian(self,symbol,geometry,num_electrons,charge):
        return qml.qchem.molecular_hamiltonian(symbol, geometry, charge=charge,\
                                               basis="STO-3G",active_electrons=num_electrons)[0]

    def hf(self,electrons,num_qubits):
        return qml.qchem.hf_state(electrons=electrons, orbitals=num_qubits)

    def circuit(self,state,weights,ops_cir):
        qml.StatePrep(state, wires=range(self.num_qubits))
        for i, excitation in enumerate(ops_cir):
            if len(excitation) == 4:
                qml.DoubleExcitation(weights[i], wires=excitation)
            else:
                qml.SingleExcitation(weights[i], wires=excitation)
        return qml.expval(self.hamiltonian)

    def circuit_h_sq(self,state,weights,ops_cir):
        qml.StatePrep(state, wires=range(self.num_qubits))
        for i, excitation in enumerate(ops_cir):
            if len(excitation) == 4:
                qml.DoubleExcitation(weights[i], wires=excitation)
            else:
                qml.SingleExcitation(weights[i], wires=excitation)
        return qml.expval(self.H_sq_ob)

    def circuit_sz(self,state,weights,ops_cir):
        qml.StatePrep(state, wires=range(self.num_qubits))
        for i, excitation in enumerate(ops_cir):
            if len(excitation) == 4:
                qml.DoubleExcitation(weights[i], wires=excitation)
            else:
                qml.SingleExcitation(weights[i], wires=excitation)
        return qml.expval(self.sz)

    def cost_fn(self,params,ops_cir):
        self.cost_hsq_fn = qml.QNode(self.circuit_h_sq, self.dev)
        return np.sqrt(self.cost_hsq_fn(self.state,params,ops_cir)-self.cost_h_fn(self.state,params,ops_cir)**2)

    def run(self,epochs=400,stepsize=0.5,cov=1e-6,print_train=True,ep=10,threshold=0.05):
        opt = qml.GradientDescentOptimizer(stepsize=stepsize)

        self.ops_cir=[]# doubles_select+singles_select

        weights = []
        self.weights=np.array(weights)
        circuit_gradient = qml.grad(self.cost_fn, argnum=0)

        E0 = self.cost_h_fn(self.state,self.weights, self.ops_cir)
        print(f"Epoch = 0, G Energy = {E0:.8f} Ha")

        i=0
        iter=[]
        cost_val=[]
        Lowest_cost=0
        self.best_weights=self.weights

        cov=cov
        E0_pev=0
        for n in range(epochs):

            Test_set=self.ops_cir+self.pool_operators
            Test_params=list(self.weights)+[0.0]*len(self.pool_operators)
            Test_params=np.array(Test_params)

            test_grads = circuit_gradient(Test_params,Test_set)
            test_grads = test_grads[len(self.weights):]
            maxpos = np.argmax(np.abs(test_grads))

            if n<= ep or np.abs(test_grads[maxpos])<threshold:

                max_op=self.pool_operators[maxpos]
                self.ops_cir.append(max_op)
                self.weights=np.append(self.weights, 0.0)

            self.weights,_ = opt.step(self.cost_fn, self.weights,self.ops_cir) # Step 11.
            E0 = self.cost_h_fn(self.state,self.weights,self.ops_cir)

            print(f"Epoch = {n+1}, G Energy ={E0:.8f} Ha, Sz ={self.Sz_fn(self.state,self.weights,self.ops_cir):.8f}")
            print("Number of gates = {}\n".format(len(self.ops_cir)))
            if np.abs(E0-E0_pev)<cov:
                break
            E0_pev=E0
        self.E0=E0
        return E0


In [111]:
L=1.5
symbol=["H", "Li"]
geometry=np.array([0.0, 0.0, -L/2, 0.0, 0.0, L/2])

charge=0
num_electrons=2
e_trial_stat_sz0={'1100000000': 1}
adapt_vqe_x_test=adapt_vqe_X_ch(e_trial_stat_sz0,num_electrons,symbol,geometry,charge)

In [112]:
adapt_vqe_x_test.run(stepsize=0.07,ep=10,threshold=0.05)

Epoch = 0, G Energy = -7.60898166 Ha
Epoch = 1, G Energy =-7.61316842 Ha, Sz =0.00000000
Number of gates = 1

Epoch = 2, G Energy =-7.61890204 Ha, Sz =0.00000000
Number of gates = 2

Epoch = 3, G Energy =-7.62166321 Ha, Sz =0.00000000
Number of gates = 3

Epoch = 4, G Energy =-7.62203569 Ha, Sz =0.00000000
Number of gates = 4

Epoch = 5, G Energy =-7.62265654 Ha, Sz =0.00000000
Number of gates = 5

Epoch = 6, G Energy =-7.62327613 Ha, Sz =0.00000000
Number of gates = 6

Epoch = 7, G Energy =-7.62387831 Ha, Sz =-0.00000000
Number of gates = 7

Epoch = 8, G Energy =-7.62462708 Ha, Sz =0.00000000
Number of gates = 8

Epoch = 9, G Energy =-7.62543952 Ha, Sz =0.00000000
Number of gates = 9

Epoch = 10, G Energy =-7.62603629 Ha, Sz =-0.00000000
Number of gates = 10

Epoch = 11, G Energy =-7.62630016 Ha, Sz =0.00000000
Number of gates = 11

Epoch = 12, G Energy =-7.62630826 Ha, Sz =0.00000000
Number of gates = 11

Epoch = 13, G Energy =-7.62421967 Ha, Sz =0.00000000
Number of gates = 11

Epoc

tensor(-7.62516801, requires_grad=True)